In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from astropy.io import ascii
from scipy.optimize import curve_fit
from astropy.timeseries import LombScargle

In [ ]:
def ringaling(phi,*c):
    # c is a numpy array containing an odd number of coeffieicnts
    # so that c[0] + c[1]*np.sin(phi) + c[2]*np.cos(phi) + c[3]*np.sin(2*phi) + c[4]*np.cos(2*phi) + .....
    c = np.array(c)
    npairs = (c.size-1)/2
    result = 0
    for i in np.arange(npairs):
        a_sin = c[((i*2)+1).astype(int)]
        a_cos = c[((i*2)+2).astype(int)]
        result = result + a_sin*np.sin(phi*(i+1)) + a_cos*np.cos(phi*(i+1))
    return result+c[0]

def binner(bins, time, flux):
    means, errors = [], []
    midpoints = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
    for i in range(len(bins)-1):
        binned_mask = (time < bins[i+1])*(time > bins[i])
        means.append(np.mean(flux[binned_mask]))
        errors.append(np.std(flux[binned_mask])/np.sqrt(len(flux[binned_mask])))
    return means, errors, midpoints

In [ ]:
#Get the data directory
cwd = os.getcwd()
data_dir = cwd.replace('Figure_5', 'Data\\')

tess = ascii.read(data_dir + 'J1407_TESS.txt')

In [ ]:
time = tess['col1']
flux = tess['col2']
eflux = tess['col3']

#Remove parts of the flux that are not trusted
mask = (time > 1602.2)*(eflux < 1000)*(flux > 100)*(time < 1623) 
time = time[mask]
flux = flux[mask]
eflux = eflux[mask]

#Remove the central section of flux by eye, as it is contaminated
shaky_mask = (time > 1609.2) * (time < 1614.5)
time_f = time[~shaky_mask]
flux = flux[~shaky_mask]
eflux = eflux[~shaky_mask]

#Normalise to the median
flux_f = flux/np.median(flux)
eflux_f = eflux/np.median(flux)

In [ ]:
#Remove the periodic signals

frequencies = 1/np.linspace(2, 5, 3000)
org_power = LombScargle(time_f, flux_f-np.mean(flux_f), dy = eflux_f).power(frequencies)
max_period = 1/frequencies[list(org_power).index(np.max(org_power))]
print("The highest power period is ", '{0:.3f}'.format(max_period), ' days.')

time_folded = time_f%max_period
means, errors, midpoints = binner(np.linspace(0, max_period, 50), time_folded, flux_f)

popt, pcov = curve_fit(ringaling, np.array(midpoints)*2*np.pi/(max_period), means, sigma = errors, absolute_sigma = True, p0 = [1,0,0, 1, 1])
phi = np.arange(200)*2*np.pi / 200

flux_32_corr = flux_f-ringaling(time_folded*2*np.pi/(max_period), *popt)

In [ ]:
#As not all signals are gone, repeat the above recipe

corr_frequencies = 1/np.linspace(2, 10, 3000)
corr_power = LombScargle(time_f, flux_32_corr-np.mean(flux_32_corr), dy = eflux_f).power(corr_frequencies)
corr_max_period = 1/corr_frequencies[list(corr_power).index(np.max(corr_power))]
print("The highest power period is ", '{0:.3f}'.format(corr_max_period), ' days.')

time_folded = time_f%corr_max_period
means, errors, midpoints = binner(np.linspace(0, corr_max_period, 50), time_folded, flux_32_corr)

popt, pcov = curve_fit(ringaling, np.array(midpoints)*2*np.pi/(corr_max_period), means, sigma = errors, absolute_sigma = True, p0 = [1,0,0, 1, 1])
phi = np.arange(200)*2*np.pi / 200

flux_32et83_corr = flux_32_corr-ringaling(time_folded*2*np.pi/(corr_max_period), *popt)

In [ ]:
#Determine the periodogram after the two corrections

final_frequencies = 1/np.linspace(2, 10, 3000)
final_power = LombScargle(time_f, flux_32et83_corr-np.mean(flux_32et83_corr), dy = eflux_f).power(final_frequencies)
final_max_period = 1/final_frequencies[list(final_power).index(np.max(final_power))]
print("The highest power period is ", '{0:.3f}'.format(final_max_period), ' days.')


import matplotlib
plt.style.use('seaborn-dark-palette')
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
matplotlib.rc('font', **font)


plt.plot(1/frequencies, org_power, label='Pre-correction TESS', color='grey')
plt.plot(1/final_frequencies, final_power, label='Post-correction TESS')
plt.legend()
plt.xlabel('Signal Period (Days)')
plt.ylabel('Signal Power')
plt.tight_layout()
plt.savefig('TESS_periodogram.pdf')
plt.show()